In [2]:
# Get the openai secret key:
import getpass

secret_key = getpass.getpass('Please enter your openai key:')

In [36]:
from openai import OpenAI
from IPython.display import display, Math, Markdown
import os

# Set up your OpenAI client
client = OpenAI(api_key=secret_key)

def get_completion(system, prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}],
        max_tokens=4000,
        temperature=1,
    )
    return response.choices[0].message.content.strip()

system = "You are a helpful assistant that can answer questions about the world."
#5 
question = "how many 4s are there in 46154144?"

standard_prompt = f"""Q: How many 2s are there in 22325672?
A: 4
Q: How many 5s are there in 51555285?
A: 5
Q: How many 9s are there in 91919929?
A: 5
{question}"""

print("Standard prompt:")
display(Markdown(get_completion(system, standard_prompt)))

#Chain of Thought
cot_prompt = f"""Q: How many 2s are there in 22325672?
Let's analyze the number step by step: 
First, we extract all digits: 2-2-3-2-5-6-7-2
Now, we count the occurrences of 2:  
- There are two 2s at the beginning.  
- One 2 appears in the middle.  
- One more 2 is at the end.  
2 + 1 + 1 = 4 twos in total.
A: 4

Q: How many 5s are there in 51555285?
Let's analyze the number step by step:
First, we extract all digits: 5-1-5-5-5-5-8-5
- There is one 5 at the beginning.  
- Four 5s appear in the middle.  
- One more 5 is at the end.  
1 + 3 + 1 = 5 fives in total.
A: 6

Q: How many 9s are there in 91919929?
Let's analyze the number step by step:
First, we extract all digits: 9-1-1-1-1-1-2-9
- There is one 9 at the beginning.    
- Two more 9s are near the end.  
1 + 1 = 2
A: 2
{question}.
Let's analyze the number step by step:"""

print("Chain of Thought prompt:")
display(Markdown(get_completion(system, cot_prompt)))

Standard prompt:


There are 3 fours in 46154144.

Chain of Thought prompt:


First, we extract all digits: 4-6-1-5-4-1-4-4.

Now, we count the occurrences of 4:  
- There is one 4 at the beginning.  
- One 4 appears in the middle.  
- Three 4s are at the end.  

So, we have 1 + 1 + 3 = 5 fours in total.

A: 5

In [32]:
import re

def evaluate_response(response, correct_answer):
    # Check if the final answer is correct
    final_answer = re.search(r'A:\s*(\d+)', response)

    if final_answer:
        answer = int(final_answer.group(1))
        is_correct = (answer == correct_answer)
    else:
        is_correct = False
    
    # Check if steps are provided
    lines = response.strip().split('\n')
    has_steps = len(lines) > 1
    
    return is_correct, has_steps

# Test the evaluation function
test_response = """Q: How many 3s are there in 3398313?  
A: Let's analyze the number step by step:  
First, we extract all digits: 3-3-9-8-3-1-3  
- There's two 3 at the beginning
- one in the middle
- and one at the end. 2 + 1 + 1 = 4 threes in total.  
A: 4 """

is_correct, has_steps = evaluate_response(test_response, 4)
print(f"Is the answer correct? {is_correct}")
print(f"Does it provide steps? {has_steps}")

Is the answer correct? True
Does it provide steps? True


In [43]:
eval_set = [
    {
        "Q": "How many 3s are there in 23323?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 2-3-3-2-3
            Now, we count the occurrences of 3:
            - There are three 3s in total.
            3 = 3 threes in total.""",
        "A": 3
    },
    {
        "Q": "How many 5s are there in 555555?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 5-5-5-5-5-5
            Now, we count the occurrences of 5:
            - There is 6 5s in row.
            6 = 6 fives in total.""",
        "A": 6
    },
    {
        "Q": "How many 1s are there in 123451?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 1-2-3-4-5-1
            Now, we count the occurrences of 1:
            - There is one 1s at the beginning.
            - One more 1s appears at the end.
            1 + 1 = 2 ones in total.""",
        "A": 2
    },
    {
        "Q": "How many 0s are there in 100101000?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 1-0-0-1-0-1-0-0-0
            Now, we count the occurrences of 0:
            - There are two 0s at the beginning.
            - One 0s appears in the middle.
            - Three more 0s are at the end.
            2 + 1 + 3 = 6 zeros in total.""",
        "A": 6
    },
    {
        "Q": "How many 7s are there in 17727177?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 1-7-7-2-7-1-7-7
            Now, we count the occurrences of 7:
            - There are two 7s at the beginning.
            - One 7s appears in the middle.
            - Two more 7s are at the end.
            2 + 1 + 2 = 5 sevens in total.""",
        "A": 5,
    },    
    {
        "Q": "How many 9s are there in 99990099?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 9-9-9-9-0-0-0-9-9
            Now, we count the occurrences of 9:
            - There are four 9s at the beginning.
            - Two more 9s appear at the end.
            4 + 2 = 6 nines in total.""",
        "A": 6
    },
    {
        "Q": "How many 8s are there in 886878?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 8-8-6-8-7-8
            Now, we count the occurrences of 8:
            - There are two 8s at the beginning.
            - There are one 8s in the middle.
            - There are one 8s at the end.
            2 + 1 + 1 = 4 eights in total.""",
        "A": 4
    },
    {
        "Q": "How many 4s are there in 441144?",
        "steps": """Let's analyze the number step by step:
            First, we extract all digits: 4-4-1-1-4-4
            Now, we count the occurrences of 4:
            - There are two 4s at the beginning.
            - Two more 4s appear at the end.
            2 + 2 = 4 fours in total.""",
        "A": 4
    },
    {
        "Q": "How many 2s are there in 2222221111145?",
        "steps": """Let's analyze the number step by step: 
            First, we extract all digits: 2-2-2-2-2-2-1-1-1-1-1-4-5
            Now, we count the occurrences of 2:
            - There are six 2s at the beginning.
            6 = 6 twos in total.
            A: 6""",
        "A": 6
    },
    {
        "Q": "How many 6s are there in 6665586666?",
        "steps": 
            """Let's analyze the number step by step: First, we extract all digits: 6-6-6-5-5-8-6-6-6-6
            Now, we count the occurrences of 6:
            - There are three 6s at the beginning.
            - Four more 6s appear later.
            3 + 4 = 7 sixes in total.
            A: 7""",
        "A": 7
    }
]


import time

def run_evaluation(prompt_type):
    correct_count = 0
    step_count = 0
    total_time = 0

    for example in eval_set:
        question = example["Q"]
        correct_answer = example["A"]

        if prompt_type == "standard":
            prompt = f"""Q: How many 2s are there in 22325672?
A: 4
Q: How many 5s are there in 51555285?
A: 5
Q: How many 9s are there in 91919929?
A: 5
{question}"""
        else:  # CoT prompt
            prompt = f"""Q: How many 2s are there in 22325672?
Let's analyze the number step by step: 
First, we extract all digits: 2-2-3-2-5-6-7-2
Now, we count the occurrences of 2:  
- There are two 2s at the beginning.  
- One 2 appears in the middle.  
- One more 2 is at the end.  
2 + 1 + 1 = 4 twos in total.
A: 4

Q: How many 5s are there in 51555285?
Let's analyze the number step by step:
First, we extract all digits: 5-1-5-5-5-5-8-5
- There is one 5 at the beginning.  
- Four 5s appear in the middle.  
- One more 5 is at the end.  
1 + 3 + 1 = 5 fives in total.
A: 6

Q: How many 9s are there in 91919929?
Let's analyze the number step by step:
First, we extract all digits: 9-1-1-1-1-1-2-9
- There is one 9 at the beginning.    
- Two more 9s are near the end.  
1 + 1 = 2
A: 2
{question}.
Let's analyze the number step by step:"""

        start_time = time.time()
        response = get_completion(system, prompt)
        end_time = time.time()

        is_correct, has_steps = evaluate_response(response, correct_answer)
        correct_count += int(is_correct)
        step_count += int(has_steps)
        total_time += (end_time - start_time)

    accuracy = correct_count / len(eval_set)
    avg_time = total_time / len(eval_set)
    step_percentage = step_count / len(eval_set) * 100

    return accuracy, avg_time, step_percentage

# Run evaluation for standard prompting
standard_accuracy, standard_avg_time, standard_step_percentage = run_evaluation("standard")

# Run evaluation for CoT prompting
cot_accuracy, cot_avg_time, cot_step_percentage = run_evaluation("cot")

# Print results
print("Standard Prompting Results:")
print(f"Accuracy: {standard_accuracy:.2%}")
print(f"Average Time: {standard_avg_time:.2f} seconds")
print(f"Percentage with Steps: {standard_step_percentage:.2f}%")

print("\nChain of Thought Prompting Results:")
print(f"Accuracy: {cot_accuracy:.2%}")
print(f"Average Time: {cot_avg_time:.2f} seconds")
print(f"Percentage with Steps: {cot_step_percentage:.2f}%")

Standard Prompting Results:
Accuracy: 0.00%
Average Time: 0.67 seconds
Percentage with Steps: 0.00%

Chain of Thought Prompting Results:
Accuracy: 80.00%
Average Time: 2.68 seconds
Percentage with Steps: 100.00%


In [ ]:
# Here's a good example where the normal prompting didn't get the correct count of digits in the number even once.
# The issue arises because the model tokenizes numbers into individual tokens in standard prompting,
# causing it to struggle with counting the occurrences of digits correctly.
# In Chain of Thought (CoT) prompting, the model receives a step-by-step breakdown, 
# which helps it better understand the task and provide accurate answers.